# Языковые модели

Языковые модели играют важную роль в системах распознавания речи, помогая создавать более грамотные и лексически корректные тексты. В данной работе мы будем изучать нграмные языковые модели, которые позволяют довольно легко оценить вероятность и правдоподобность текста.

В нграмной языковой модели, нграм - это последовательность из n слов в тексте. Например, в предложении "по-моему мы сэкономим уйму времени если я сойду с ума прямо сейчас", биграмами будут "по-моему мы", "мы сэкономим", "сэкономим уйму" итд. Языковые модели оценивают вероятность появления последовательности слов, исходя из статистики появления каждого из нграм в обучающей выборке.

Порядком (order) нграм языковой модели называют максимальную длину нграм, которую учитывает модель. 

Практическая работа разделена на 2 части: 
1. Построение нграмой языковой модели - основная часть, 12 баллов
1. Предсказание с помощью языковой модели - дополнительная часть, 4 балла



Полезные сслыки:
* arpa формат - https://cmusphinx.github.io/wiki/arpaformat/
* обучающие материалы - https://pages.ucsd.edu/~rlevy/teaching/2015winter/lign165/lectures/lecture13/lecture13_ngrams_with_SRILM.pdf
* обучающие материалы.2 - https://cjlise.github.io/machine-learning/N-Gram-Language-Model/

In [1]:
import numpy as np
from collections import defaultdict
from typing import List, Dict, Tuple

# 1. Построение нграмной языковой модели. (12 баллов)


Вероятность текста с помощью нграмной языковой модели можно вычислить по формуле: 
$$ P(w_1, w_2, .., w_n) = {\prod{{P_{i=0}^{n}(w_i| w_{i-order}, .., w_{i-1})}}} $$

В простом виде, при обучении нграмной языковой модели, чтобы рассчитать условную вероятность каждой нграмы, используется формула, основанная на количестве появлений нграмы в обучающей выборке. Формула выглядит следующим образом:
$$ P(w_i| w_{i-order}, .., w_{i-1}) = {{count(w_{i-order}, .., w_{i})} \over {count(w_{i-order},..., w_{i-1})}} $$


In [3]:
# в первую очередь нам понадобится подсчитать статистику по обучающей выборке 
def count_ngrams(train_text: List[str], order=3, bos=True, eos=True) -> Dict[Tuple[str], int]:
    
    BOS = "<s>"
    EOS = "</s>"
    
    ngrams = defaultdict(int)
    
    for line in train_text:
        
        words = line.split()
        if bos: words.insert(0, BOS)
        if eos: words.append(EOS)
        
        # TODO: for order then word 
        for word_idx in range(1, len(words) + 1):
            for order_idx in range(0, order):
                if order_idx < word_idx:
                    ngrams[tuple(words[word_idx-order_idx-1:word_idx])] += 1
    
    return dict(ngrams)

In [4]:
def test_count_ngrams():
    assert count_ngrams(['привет привет как дела'], order=1, bos=True, eos=True) == {
        ('<s>',): 1, 
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1, 
        ('</s>',): 1
    }
    assert count_ngrams(['привет привет как дела'], order=1, bos=False, eos=True) == {
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1, 
        ('</s>',): 1
    }
    assert count_ngrams(['привет привет как дела'], order=1, bos=False, eos=False) == {
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1
    }
    assert count_ngrams(['привет привет как дела'], order=2, bos=False, eos=False) == {
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1,
        ('привет', 'привет'): 1,
        ('привет', 'как'): 1,
        ('как', 'дела'): 1
    }    
    assert count_ngrams(['привет ' * 6], order=2, bos=False, eos=False) == {
        ('привет',): 6, 
        ('привет', 'привет'): 5
    }
    result = count_ngrams(['практическое сентября',
                           'второе практическое занятие пройдет в офлайне 32 сентября в 12 часов 32 минуты',
                           'в офлайне в 32 12'], order=5)
    assert result[('<s>',)] == 3
    assert result[('32',)] == 3
    assert result[('<s>', 'в', 'офлайне', 'в', '32')] == 1
    assert result[('офлайне', 'в', '32', '12', '</s>')] == 1
    print('Test 1a passed')
    
    
test_count_ngrams()  

Test 1a passed



Простой подход к вычислению вероятностей через количество нграм имеет существенный недостаток. Если в тексте встретится нграмма, которой не было в обучающей выборке, то вероятность всего текста будет равна нулю. 

Чтобы избежать данного недостатка, вводится специальное сглаживание - [сглаживание Лапласа](https://en.wikipedia.org/wiki/Additive_smoothing). Данная техника позволяет учитывать нграмы, не встретившиеся в обучающей выборке, и при этом не делает вероятность текста равной нулю.

Формула сглаживания Лапласа выглядит следующим образом:

$$ P(w_i| w_{i-order}, .., w_{i-1}) = {{count(w_{i-order}, .., w_{i}) + k} \over {count(w_{i-order},..., w_{i-1}) + k*V}} $$

Здесь V - количество слов в словаре, а k - гиперпараметр, который контролирует меру сглаживания. Как правило, значение k выбирается экспериментально, чтобы найти оптимальный баланс между учетом редких нграм и сохранением вероятности для часто встречающихся нграм.

In [21]:
# функция подсчета вероятности через количество со сглаживанием Лапласа
def calculate_ngram_prob(ngram: Tuple[str], counts: Dict[Tuple[str], int], V=None, k=0) -> float:
    # подсчитывет ngram со сглаживанием Лапласа
    if V is None:
        V = len(set(word for words in counts for word in words))
    counts = defaultdict(int, counts)
    numerator = counts[ngram] + k
    if len(ngram) != 1:
        denominator = counts[ngram[:-1]] + k * V
    else:
        denominator = sum(map(lambda x: x[1], filter(lambda x: len(x[0]) == 1, counts.items()))) + k * V
    prob = numerator / denominator
    return prob

In [23]:
def test_calculate_ngram_prob():
    counts = count_ngrams(['практическое сентября',
                           'второе практическое занятие в офлайне 32 сентября в 12 часов 32 минуты',
                           'в офлайне в 32 12'], order=4)
    assert calculate_ngram_prob(('в', 'офлайне'), counts) == 0.5
    assert calculate_ngram_prob(('в', ), counts) == 4/25
    assert calculate_ngram_prob(('в', ), counts, k=0.5) == (4+0.5)/(25+0.5*12)
    assert calculate_ngram_prob(('в', 'офлайне', 'в', '32'), counts) == 1.0
    assert calculate_ngram_prob(('в', 'офлайне'), counts, k=1) == 0.1875
    assert calculate_ngram_prob(('в', 'офлайне'), counts, k=0.5) == 0.25
    assert calculate_ngram_prob(('в', 'онлайне'), counts, k=0) == 0.0
    assert calculate_ngram_prob(('в', 'онлайне'), counts, k=1) == 0.0625
    assert calculate_ngram_prob(('в', 'офлайне'), counts, k=0.5) == 0.25

    print("Test 1.b passed")
    

test_calculate_ngram_prob() 

Test 1.b passed


Основной метрикой язковых моделей является перплексия. 

Перплексия  — безразмерная величина, мера того, насколько хорошо распределение вероятностей предсказывает выборку. Низкий показатель перплексии указывает на то, что распределение вероятности хорошо предсказывает выборку.

$$ ppl = {P(w_1, w_2 ,..., w_N)^{- {1} \over {N}}} $$


In [24]:
# Языковая модель 
class NgramLM:
    def __init__(self, order=3, bos=True, eos=True, k=1, predefined_vocab=None):
        self.order = order
        self.eos = eos
        self.bos = bos
        self.k = k
        self.vocab = predefined_vocab
        self.ngrams_count = None
        
    @property
    def V(self) -> int:
        return len(self.vocab)
    
    def fit(self, train_text: List[str]) -> None:
        self.ngrams_count = defaultdict(int, count_ngrams(train_text, self.order, self.bos, self.eos))
        self.vocab = set(word for words in self.ngrams_count for word in words)     
    
    def predict_ngram_log_proba(self, ngram: Tuple[str]) -> float:
        # считаем логарифм вероятности конкретной нграмы
        return np.log(calculate_ngram_prob(ngram, self.ngrams_count, self.V, self.k))
           
    def predict_log_proba(self, words: List[str]) -> float:
        if self.bos:
            words = ['<s>'] + words
        if self.eos:
            words = words + ['</s>']
        logprob = []
        # применяем chain rule, чтобы посчитать логарифм вероятности всей строки
        logprob.append(self.predict_ngram_log_proba(tuple([words[0]])))
        for word_idx in range(len(words) - self.order + 1):
            logprob.append(self.predict_ngram_log_proba(tuple(words[word_idx:word_idx+self.order])))  
        return sum(logprob) / len(logprob)
        
    def ppl(self, text: List[str]) -> float:
        # подсчет перплексии
        # Для того, чтобы ваш код был численно стабильным, 
        #    не считайте формулу напрямую, а воспользуйтесь переходом к логарифмам вероятностей
        words = text[0].split()
        perplexity = np.exp(-self.predict_log_proba(words))
        return perplexity

In [26]:
def test_lm():
    train_data = ["по-моему мы сэкономим уйму времени если я сойду с ума прямо сейчас",
                  "если я сойду с ума прямо сейчас по-моему мы сэкономим уйму времени",
                  "мы сэкономим уйму времени если я сейчас сойду с ума по-моему"]
    global lm
    lm = NgramLM(order=2)
    lm.fit(train_data)
    assert lm.V == 14
    assert np.isclose(lm.predict_log_proba(['мы']), lm.predict_log_proba(["если"]))
    assert lm.predict_log_proba(["по-моему"]) > lm.predict_log_proba(["если"]) 
    
    gt = ((3+1)/(41 + 14) * 1/(3+14))**(-1/2)
    ppl = lm.ppl([''])
    assert  np.isclose(ppl, gt), f"{ppl=} {gt=}"
    
    gt = ((3+1)/(41 + 14) * 1/(3+14) * 1/(14)) ** (-1/3)
    ppl = lm.ppl(['ЧТО'])
    assert  np.isclose(ppl, gt), f"{ppl=} {gt=}"
    
    test_data = ["по-моему если я прямо сейчас сойду с ума мы сэкономим уйму времени"]
    ppl = lm.ppl(test_data)
    assert round(ppl, 2) == 7.33, f"{ppl}"
test_lm()

# 2. Предсказания с помощью языковой модели (4 балла)

In [27]:
def predict_next_word(lm: NgramLM, prefix: List[str], topk=4):
    # TODO реализуйте функцию, которая предсказывает продолжение фразы. 
    # верните topk наиболее вероятных продолжений фразы prefix 
    pred_scores = {}
    for word in lm.vocab:
        pred_scores[word] = lm.ppl([prefix[0] + ' ' + word])
    return sorted(pred_scores.items(), key=lambda x: x[1])[:topk]
    

Обучите языковую модель на всем тексте из этой лабораторной работы (не забудьте заранее нормализовать текст).

Что предскажет модель по префиксам `по-моему`, `сэкономим`, `нграм` и `вероятности`? 

In [28]:
train_data = ["по-моему мы сэкономим уйму времени если я сойду с ума прямо сейчас",
              "если я сойду с ума прямо сейчас по-моему мы сэкономим уйму времени",
              "мы сэкономим уйму времени если я сейчас сойду с ума по-моему",'практическое сентября',
              "второе практическое занятие пройдет в офлайне тридцать второго сентября в двенадцать часов тридцать две минуты",
              "в офлайне в тридцать два двенадцать"]
lm = NgramLM(order=2)
lm.fit(train_data)
predict_next_word(lm, ["по-моему"])

[('мы', 16.616699746389855),
 ('минуты', 18.08533939518749),
 ('двенадцать', 18.23927105604001),
 ('сентября', 18.23927105604001)]

In [29]:
predict_next_word(lm, ["сэкономим"][0]) 

[('ума', 18.38940117441857),
 ('минуты', 21.50721428599597),
 ('двенадцать', 21.690270912305976),
 ('сентября', 21.690270912305976)]

In [30]:
predict_next_word(lm, ["нграм"][0]) 

[('минуты', 20.966852818365762),
 ('двенадцать', 21.14531020899899),
 ('сентября', 21.14531020899899),
 ('времени', 21.319360362378184)]

In [31]:
predict_next_word(lm, ["вероятности"][0]) 

[('двенадцать', 15.45955717155684),
 ('офлайне', 16.612375295861984),
 ('минуты', 18.22945707828453),
 ('сентября', 18.384615383206736)]